In [1]:
import os

from dotenv import load_dotenv, find_dotenv
from openai import OpenAI

_ = load_dotenv(find_dotenv())
llm_api_key = os.environ['DEEPSEEK_API_KEY']
MODEL = "deepseek-chat"
client = OpenAI(api_key=llm_api_key, base_url="https://api.deepseek.com")

In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import random

loader = PyPDFLoader("../data/Kompendium1.pdf")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=20)
documents = loader.load_and_split(text_splitter)

random_numbers = random.sample(range(6, 400), 20)
docs_subset = [documents[i] for i in random_numbers]

In [3]:
documents[0].page_content

'SOZIALVERSICHERUNGSANSTALT\nDER BAUERN\nKompendium für die\nArbeitsmedizin\nin der\nLand- und\nForstwirtschaft\nWien, September 1998'

In [7]:
system_prompt = "You are a helpful assistant. Your role is to generate a single question in German that can be answered from the CONTEXT provided by the user. This will be used to evaluate a RAG app. The response should only be the question."

testset = {"question": [], "context": []}
for doc in docs_subset:
    context = doc.page_content
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": context},
        ],
        stream=False
    )
    testset["context"].append(doc.page_content)
    testset["question"].append(response.choices[0].message.content)


In [8]:
import pandas as pd

df = pd.DataFrame.from_dict(testset)
df

,question,context
0,Wie unterstützt die SVB vermutlich betroffene ...,„Zielgruppenmodell“ und die \n● Möglichkeiten ...
1,Was sind die Schritte zur Durchführung eines P...,a) Pricktest\nZuerst wird 1 Tropfen der T estl...
2,Warum ist Flüssigmist gefährlicher einzustufen...,Ausführlicheres ist dem Kapitel „Stäube und Al...
3,In welchem Jahr wurde die Berufskrankheit 2110...,● Wirbelsäule: degenerative Veränderung der Wi...
4,Wie viele Prozent der Bäuerinnen und Bauern be...,Zusammenfassung der wesentlichen Ergebnisse de...
5,In welchem Jahr veröffentlichte H. Seyfried da...,"2. H. Pozenel, H. Seyfried (1993), Herz- Kr ei..."
6,Was sind mögliche Anlässe für eine Sicherheits...,Insgesamt werden derzeit 12 Hauptgruppen chif ...
7,"Was bedeutet der Begriff ""fortpflanzungsgefähr...","12. „krebserzeugend“,\nwenn sie durch Einatmen..."
8,Welche gesundheitlichen Auswirkungen können du...,im Liegen über die Rückenfläche oder \nim Sitz...
9,In wie vielen Proben wurde Escheria coli nachg...,● bei 20 Proben das Vorkommen von Escheria col...


In [6]:
df.to_csv("../data/testset.csv", encoding="utf-8", index=False)